In [ ]:
!pip install -qU datasets accelerate
!pip install -qU transformers[torch]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
df = pd.read_csv("./Filtered_labelled_ge_transcripts (1).csv")
encoder = preprocessing.LabelEncoder()
df.rename(columns = {'Label': 'labels'}, inplace=True)
#print(df.head())
encoder.fit(df['labels'])
df['labels']=encoder.transform(df['labels'])
print(df.head())
X_train, X_test, y_train, y_test = train_test_split(df['transcripts'], df['labels'], test_size=0.2, random_state=20)
train = pd.DataFrame([X_train, y_train]).T
train.set_index('transcripts').dropna().to_csv('train.csv')

test = pd.DataFrame([X_test, y_test]).T
test.set_index('transcripts').dropna().to_csv('test.csv')


   Unnamed: 0      videoId                                              title  \
0           0  joLk0IPEniA  WILL RAFALE BE A FACTOR THIS ELECTION? | FACEO...   
1           1  Gc1tf5U7BuU  BJP Manifesto Promises To Secure &amp; Protect...   
2           2  1BAgeFCwR3o  Reporters Project: Andhra Pradesh’s Politics O...   
3           3  HKCBxHhnfC4  Reporters Project: Giving Voice To The Real Is...   
4           4  iLlUz3a4zDg  “I Definitely Don&#39;t Fight A War To Lose” –...   

                                         description          published_at  \
0  RafaleTwist DAY BEFORE POLLS, RAFALE BLOW TO G...  2019-04-10T16:41:10Z   
1  BattleOf2019 #ElectionsWithNews18 #LokSabhaEle...  2019-04-08T14:19:22Z   
2  BattleOf2019 #ElectionsWithNews18 #LokSabhaEle...  2019-04-07T09:22:24Z   
3  BattleOf2019 #ElectionsWithNews18 #LokSabhaEle...  2019-04-07T09:20:19Z   
4  On Off Centre Anuradha SenGupta is talking to ...  2019-04-06T10:28:52Z   

      channel                               

In [ ]:
from datasets import load_dataset, load_metric
import numpy as np
model_metric = load_metric('accuracy')

<ipython-input-3-0ac10517b774>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  model_metric = load_metric('accuracy')


In [ ]:
def compute_metric(model_pred):
  print(model_pred)
  preds, labels = model_pred
  # else: preds, labels, acc = model_pred
  preds = np.argmax(preds, -1)
  return model_metric.compute(predictions=preds, references=labels)

In [ ]:
dataset = load_dataset("csv", data_files="train.csv")
test_set = load_dataset("csv", data_files="test.csv")
print(dataset['train']['transcripts'][10])
set(dataset['train']['labels']), dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

registered case 110 tribals attacking police CRPF forest officials Wednesday tribals pelted stones officials amravati injuring around 45 Forest Department officials 20 policemen violence erupted tribals moved fact tribals moved core area reserve [Music] Beshara joining us phone line show case registered 110 tribals attacking government officers share details please that's right incident comes right tribals got agitated goes back 2002 rehabilitation process going many rebels shifted Accola district corporate area since new residence near semi-open town forced change lifestyle look new occupation recently Africa started moving back villages falls tiger suicide danger life living designated reserve also punishable offense Oh Tuesday last see officers concerned reached village force tribal villagers attacked shock weapons access Spears matter entire incident least 15 people injured amravati dozen vehicles destroyed tribals section 144 also enforced area police finally taken action hundred 

({0, 1, 2},
 DatasetDict({
     train: Dataset({
         features: ['transcripts', 'labels'],
         num_rows: 1011
     })
 }))

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest",num_labels = 3,ignore_mismatched_sizes=True)

def tokenize_fn(samples):
  return tokenizer(samples['transcripts'], max_length=512, truncation=True, padding='max_length')

tokenized_train_set = dataset['train'].map(tokenize_fn, batched=True)
tokenized_test_set = test_set['train'].map(tokenize_fn, batched=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/1011 [00:00<?, ? examples/s]

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    push_to_hub=False,
    label_names=['class'],
    fp16=True,
    save_strategy='no',
    do_eval=True,
    # eval_steps=10,
    logging_strategy="epoch"
)
# fp16 => For using GPU

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs.loss
        # print('----------------------------------', model(**inputs), '---------------------------------')
        return (loss, outputs) if return_outputs else loss


trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized_train_set,
                  eval_dataset=tokenized_test_set,
                  compute_metrics=compute_metric
                  )

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.864700,No log
2,0.745100,No log
3,0.620100,No log
4,0.484000,No log
5,0.341400,No log
6,0.204500,No log
7,0.145100,No log
8,0.081500,No log
9,0.045600,No log
10,0.026200,No log


TrainOutput(global_step=2540, training_loss=0.1813997379672809, metrics={'train_runtime': 902.8276, 'train_samples_per_second': 22.396, 'train_steps_per_second': 2.813, 'total_flos': 5320153306460160.0, 'train_loss': 0.1813997379672809, 'epoch': 20.0})

In [ ]:
pred = trainer.predict(tokenized_train_set)
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)
accuracy_score(a, y_train)

<ipython-input-9-7743a01b91be>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)


0.9990108803165183

In [ ]:
pred = trainer.predict(tokenized_test_set)
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)
accuracy_score(a, y_test)

<ipython-input-10-28fc1102d561>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a = F.softmax(torch.tensor(pred.predictions[1], dtype=torch.float)).argmax(-1)


0.7312252964426877

In [ ]:
trainer.save_model('Saved_model_ROBERTA')

In [ ]:
!zip -r /content/Saved_model_ROBERTA.zip /content/Saved_model_ROBERTA

  adding: content/Saved_model_ROBERTA/ (stored 0%)
  adding: content/Saved_model_ROBERTA/training_args.bin (deflated 51%)
  adding: content/Saved_model_ROBERTA/config.json (deflated 51%)
  adding: content/Saved_model_ROBERTA/model.safetensors (deflated 7%)


In [ ]:
!mv '/content/Saved_model_ROBERTA.zip' '/content/drive/MyDrive'